# Sample for Inference on kafka image events from Minio

The notebook shows how to use KFServing SDK to create InferenceService with transformer, predictor.

In [1]:
import os,sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TransformerSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1Container
from kubernetes.client import V1ResourceRequirements
import kubernetes.client
import os
import requests 
import json
import numpy as np

## Define InferenceService with Transformer

Add predictor and transformer on the endpoint spec

In [110]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='s3://mnist/model-v1/export',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))),
                          transformer=V1alpha2TransformerSpec(
                            min_replicas=1,
                            custom=V1alpha2CustomSpec(
                              container=V1Container(
                              image='yuzisun/mnist-transformer:latest',
                              name='kfserving-container',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'})))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='mnist'),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

## Create InferenceService with Transformer

Call KFServingClient to create InferenceService.

In [111]:
KFServing = KFServingClient()
KFServing.create(isvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'InferenceService',
 'metadata': {'creationTimestamp': '2019-11-19T22:59:10Z',
  'generation': 1,
  'name': 'mnist',
  'namespace': 'default',
  'resourceVersion': '162221',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/default/inferenceservices/mnist',
  'uid': '87f9afaf-8595-4578-a158-2e8a59f51cbf'},
 'spec': {'default': {'predictor': {'minReplicas': 1,
    'tensorflow': {'resources': {'limits': {'cpu': '100m', 'memory': '1Gi'},
      'requests': {'cpu': '100m', 'memory': '1Gi'}},
     'runtimeVersion': '1.14.0',
     'storageUri': 's3://mnist/model-v1'}},
   'transformer': {'custom': {'container': {'image': 'yuzisun/mnist-transformer:latest',
      'name': 'kfserving-container',
      'resources': {'limits': {'cpu': '100m', 'memory': '1Gi'},
       'requests': {'cpu': '100m', 'memory': '1Gi'}}}},
    'minReplicas': 1}}},
 'status': {}}

## Check the InferenceService

In [112]:
KFServing.get('mnist', watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
mnist                False                                                                                        
mnist                False                                                                                        
mnist                False                                                                                        
mnist                True       100                             http://mnist.default.example.com/v1/models/mnist  


## Predict the image

In [ ]:
api_instance = kubernetes.client.CoreV1Api(kubernetes.client.ApiClient())
service = api_instance.read_namespaced_service("istio-ingressgateway", "istio-system", exact='true')
#cluster_ip = "service.status.load_balancer.ingress[0].ip"
cluster_ip = "127.0.0.1:8080"

In [ ]:
url = "http://" + cluster_ip + "/v1/models/mnist/metadata"
headers = { 'Host': 'mnist-predictor-default.default.example.com' }
response = requests.get(url, headers=headers)
print(response.content.decode('utf-8'))

## Delete the InferenceService

In [ ]:
KFServing.delete('mnist')